In [416]:
import csv
import pprint

# with open("skill_levels.csv", encoding="utf8") as f:
#     skill_table = [x for x in csv.DictReader(f)]

with open("armor_base.csv", encoding="utf8") as f:
    armor_table = [x for x in csv.DictReader(f)]

with open("armor_skills_ext.csv", encoding="utf8") as f:
    armor_skills_table = dict((x["base_name_en"], x) for x in csv.DictReader(f))

with open("charm_base.csv", encoding="utf8") as f:
    charm_table = [x for x in csv.DictReader(f)]

print(armor_table[-1])
print(armor_skills_table["Leather Headgear"])
print(charm_table[0])


{'id': '1470', 'name_en': 'Wyverian Ears α+', 'rarity': '9', 'type': 'head', 'gender': 'both', 'slot_1': '4', 'slot_2': '0', 'slot_3': '0', 'defense_base': '140', 'defense_max': '178', 'defense_augment_max': '198', 'defense_fire': '1', 'defense_water': '1', 'defense_thunder': '1', 'defense_ice': '1', 'defense_dragon': '5'}
{'base_name_en': 'Leather Headgear', 'skill1_name': 'Hunger Resistance', 'skill1_level': '1', 'skill2_name': '', 'skill2_level': ''}
{'id': '1', 'name_en': 'Poison Charm I', 'previous_en': '', 'rarity': '3', 'skill1_name': 'Poison Resistance', 'skill1_level': '1', 'skill2_name': '', 'skill2_level': ''}


In [417]:
# join and reformat.
equipment_table = []
for armor in armor_table:
    if int(armor["rarity"]) < 8:
        continue
    equipment_table.append({
        "name_en": armor["name_en"],
        "rarity": int(armor["rarity"]),
        "type": armor["type"],
        "slot_1": int(armor["slot_1"]),
        "slot_2": int(armor["slot_2"]),
        "slot_3": int(armor["slot_3"]),
        "defense_base": int(armor["defense_base"]),
        "defense_max": int(armor["defense_max"]),
        "defense_augment_max": int(armor["defense_augment_max"]),
        # and whatever ig

        "skill1_name": armor_skills_table[armor["name_en"]]["skill1_name"],
        "skill1_level": int(armor_skills_table[armor["name_en"]]["skill1_level"]) if armor_skills_table[armor["name_en"]]["skill1_name"] != "" else 0,
        "skill2_name": armor_skills_table[armor["name_en"]]["skill2_name"],
        "skill2_level": int(armor_skills_table[armor["name_en"]]["skill2_level"]) if armor_skills_table[armor["name_en"]]["skill2_name"] != "" else 0,
    })
for charm in charm_table:
    equipment_table.append({
        "name_en": charm["name_en"],
        "rarity": int(charm["rarity"]),
        "type": "charm",
        "slot_1": 0,
        "slot_2": 0,
        "slot_3": 0,
        "defense_base": 0,
        "defense_max": 0,
        "defense_augment_max": 0,
        # and whatever ig

        "skill1_name": charm["skill1_name"],
        "skill1_level": int(charm["skill1_level"]) if charm["skill1_name"] != "" else 0,
        "skill2_name": charm["skill2_name"],
        "skill2_level": int(charm["skill2_level"]) if charm["skill2_name"] != "" else 0,
    })

In [418]:
skill_list = tuple(sorted(set(armor["skill1_name"] for armor in armor_skills_table.values() if armor["skill1_name"] != "").union(set(armor["skill2_name"] for armor in armor_skills_table.values() if armor["skill2_name"] != ""))))
for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
    out = tuple(skill for skill in skill_list if skill[0] == letter)
    if len(out) > 0:
        print(out)

('Affinity Sliding', 'Agitator', 'Airborne', 'Aquatic/Polar Mobility', 'Artillery', 'Attack Boost')
('BBQ Master', 'Blast Attack', 'Blast Resistance', 'Bleeding Resistance', 'Blight Resistance', 'Blindsider', 'Bombardier', 'Botanist')
('Capacity Boost', 'Carving Pro', 'Cliffhanger', 'Coalescence', 'Constitution', 'Critical Boost', 'Critical Draw', 'Critical Eye')
('Defense Boost', 'Detector', 'Divine Blessing', 'Dragon Attack', 'Dragon Resistance', 'Dungmaster')
('Earplugs', 'Effluvia Resistance', 'Effluvial Expert', 'Elderseal Boost', 'Entomologist', 'Evade Extender', 'Evade Window')
('Fire Attack', 'Fire Resistance', 'Flinch Free', 'Focus', "Forager's Luck", 'Fortify', 'Free Elem/Ammo Up', 'Free Meal')
('Geologist', 'Guard')
('Handicraft', 'Health Boost', 'Heat Guard', 'Heavy Artillery', 'Heroics', 'Honey Hunter', 'Horn Maestro', 'Hunger Resistance')
('Ice Attack', 'Ice Resistance', 'Intimidator', 'Iron Skin', 'Item Prolonger')
('Jump Master',)
('Latent Power', 'Leap of Faith')
('Mar

In [419]:
desired_skills = {
    "Artillery": 3,
    "Capacity Boost": 1,
    "Evade Extender": 3,
    "Quick Sheath": 2,
    "Divine Blessing": 3,
    "Slinger Capacity": 3,
}

In [420]:
import numpy

# maximize this
defense_row = numpy.fromiter((equipment["defense_base"] for equipment in equipment_table), int)

# there must be exactly one of each
head_row = numpy.fromiter((equipment["type"] == "head" for equipment in equipment_table), int)
chest_row = numpy.fromiter((equipment["type"] == "chest" for equipment in equipment_table), int)
arms_row = numpy.fromiter((equipment["type"] == "arms" for equipment in equipment_table), int)
waist_row = numpy.fromiter((equipment["type"] == "waist" for equipment in equipment_table), int)
legs_row = numpy.fromiter((equipment["type"] == "legs" for equipment in equipment_table), int)
charm_row = numpy.fromiter((equipment["type"] == "charm" for equipment in equipment_table), int)

equipment_matrix = numpy.vstack((head_row, chest_row, arms_row, waist_row, legs_row, charm_row))
equipment_upper_bound = numpy.ones(6)
equipment_lower_bound = numpy.ones(6)

# there must be the user selected skills
def get_skill_level(equipment, skill):
    if equipment["skill1_name"] == skill:
        return equipment["skill1_level"]
    if equipment["skill2_name"] == skill:
        return equipment["skill2_level"]
    return 0

skill_matrix = numpy.vstack(
    tuple(
        numpy.fromiter((get_skill_level(equipment, skill) for equipment in equipment_table), int)
        for skill in skill_list
    )
)
skill_lower_bound = numpy.fromiter((desired_skills[skill] if skill in desired_skills else 0 for skill in skill_list), int)

In [421]:
from scipy import optimize

res = optimize.linprog(
    c=-defense_row,
    A_ub=numpy.concatenate((equipment_matrix, -equipment_matrix, -skill_matrix)),
    b_ub=numpy.concatenate((equipment_upper_bound, -equipment_lower_bound, -skill_lower_bound)),
    bounds=(0, 1),
    integrality=1,
)

output = list()
for equipment, delta in zip(equipment_table, res.x):
    if delta > 0:
        output.append(equipment)
print("\n".join(equipment["name_en"] for equipment in output))

skills = dict()
for equipment in output:
    if equipment["skill1_name"] != "":
        if equipment["skill1_name"] not in skills:
            skills[equipment["skill1_name"]] = 0
        skills[equipment["skill1_name"]] += int(equipment["skill1_level"])
    if equipment["skill2_name"] != "":
        if equipment["skill2_name"] not in skills:
            skills[equipment["skill2_name"]] = 0
        skills[equipment["skill2_name"]] += int(equipment["skill2_level"])
pprint.pprint(skills)

print(f"defense: {sum(equipment['defense_base'] for equipment in output)}")

Dodogama Greaves α+
Rimeguard Mail α+
Brachydium Braces α+
Tentacle Coil  γ+
Azure Age Hair α+
Blessing Charm III
{'Agitator': 2,
 'Artillery': 3,
 'Blast Attack': 1,
 'Capacity Boost': 1,
 'Critical Draw': 1,
 'Divine Blessing': 3,
 'Evade Extender': 3,
 'Quick Sheath': 2,
 'Recovery Speed': 2,
 'Slinger Capacity': 3,
 'Weakness Exploit': 1}
defense: 790


In [422]:
import random
res = optimize.linprog(
    c=-numpy.fromiter((random.random() for _ in defense_row), float),
    A_ub=numpy.concatenate((equipment_matrix, -equipment_matrix, -skill_matrix)),
    b_ub=numpy.concatenate((equipment_upper_bound, -equipment_lower_bound, -skill_lower_bound)),
    bounds=(0, 1),
    integrality=1,
)

output = list()
for equipment, delta in zip(equipment_table, res.x):
    if delta > 0:
        output.append(equipment)
print("\n".join(equipment["name_en"] for equipment in output))

skills = dict()
for equipment in output:
    if equipment["skill1_name"] != "":
        if equipment["skill1_name"] not in skills:
            skills[equipment["skill1_name"]] = 0
        skills[equipment["skill1_name"]] += int(equipment["skill1_level"])
    if equipment["skill2_name"] != "":
        if equipment["skill2_name"] not in skills:
            skills[equipment["skill2_name"]] = 0
        skills[equipment["skill2_name"]] += int(equipment["skill2_level"])
pprint.pprint(skills)

print(f"defense: {sum(equipment['defense_base'] for equipment in output)}")

Dodogama Greaves β+
Rimeguard Mail α+
Golden Lunecoil α+
Brachydium Braces α+
Azure Age Hair α+
Leaping Charm III
{'Agitator': 2,
 'Artillery': 3,
 'Capacity Boost': 1,
 'Critical Draw': 1,
 'Divine Blessing': 3,
 'Evade Extender': 3,
 'Health Boost': 1,
 'Quick Sheath': 2,
 'Slinger Capacity': 3,
 'Weakness Exploit': 1}
defense: 776


In [423]:
possible_skills = dict()
letter = None
for new_skill in skill_list:
    for new_skill_level in range(1, 5):
        new_skill_lower_bound = numpy.fromiter((new_skill_level if new_skill == skill else 0 for skill in skill_list), int)

        res = optimize.linprog(
            c=numpy.zeros_like(defense_row),
            A_ub=numpy.concatenate((equipment_matrix, -equipment_matrix, -skill_matrix, -skill_matrix)),
            b_ub=numpy.concatenate((equipment_upper_bound, -equipment_lower_bound, -skill_lower_bound, -new_skill_lower_bound)),
            bounds=(0, 1),
            integrality=1,
        )

        if res.status == 0:
            possible_skills[new_skill] = new_skill_level

    if new_skill in possible_skills:
        if new_skill[0] != letter:
            if letter != None:
                print()
            letter = new_skill[0]
        print((new_skill, possible_skills[new_skill]), end=", ")

('Agitator', 2), ('Artillery', 3), 
('Blast Attack', 1), 
('Capacity Boost', 1), ('Critical Draw', 1), 
('Divine Blessing', 3), 
('Evade Extender', 3), 
('Health Boost', 1), 
('Quick Sheath', 2), 
('Recovery Speed', 2), 
('Slinger Capacity', 3), 
('Weakness Exploit', 1), 